# 4-D STEM Analysis Using Pyxem 


## Data Inspection- Preprocessing - Unsupervised ML - Lazy Processing - Orientation Analysis 


### Carter Francis | University of Wisconsin Madison | May 24th 2023

Pyxem Introduction:
-------------------

`Pyxem` was first started in 2016 by Duncan Johnstone.  Since then it has been continually developed alongside hyperspy. Below is a very simplified dependancy tree for `pyxem`. We inherit quite a bit of functionality from upstream packages and ascribe strongly to the ideal that if we can upstream code to make it available to a wider audience we should!

We are always looking for more people to join our team [here](https://github.com/pyxem/pyxem)!  

<img style="left" src="imgs/DependancyTree.svg">

### Our Focus/ Goals:

1. Provide Scalable analysis for pixelated(mostly 4-D STEM) diffraction 
    - Pyxem (and hyperspy) scales from single core --> Multi-core single machiene --> Multicore Distributed Computing!
    - Fast I-O allows streaming and processing of TB sized datasets in under a ___minute___!
    - Hyperspy is ___Fast___! I mean like really fast. If you don't believe me try running using the dask distributed backend. 
2. Provide End to End workflows without limiting functionality.
    - A focus on documentation and example notebooks keeping the internal `pyxem` code simple and easy to maintain and grow.
3. Testing Testing Testing!
    - Pyxem is focused on test driven development which limits the number of bugs and helps us to understand why bugs arise when they do
    - While not perfect this helps us to know that updates won't cause functionality to fail.
4. Learning and Teaching!
    - Drop by and say hi on github.  Make an [issue](https://github.com/pyxem/pyxem/issues) for a feature you would like, add some code you find helpful. 
    - Even if you are just trying something out or need help we are always happy to help!

Data Introduction
-----------------
This data is a set of MgO nanocrystals on a lacy Carbon sample.  It is a pretty (very small) 4-D STEM dataset that I could load into RAM. But I personally still like to run everything lazily (and using the distributed backend) for a couple of reasons:

1. I'm Lazy (why shouldn't my data be):
    - I like things to load imediately and don't like waiting around
    - Lazy makes things like running in parallel EASY
2. Lazy means better parallelization (and it's Fast!):
    - Lazy data is already set up to run in parallel so you get better control
    - I love the dask-dashboard (and you should too)
3. One workflow, Any Size of Data
    - You can very easily take the same code. Move it to a cluster or a HPC cluster :)
    - Lazy workflows mean faster iteration, faster discovery which means more experiments. 


## 4-D STEM (What is it?)

<img style="left" src="imgs/4DSTEM.gif">

## Contents
<a id='Index'></a>

0. [Import pyxem and other required libraries](#Section0)
1. [Loading and Inspection](#Section1)
2. [Alignment & Calibration](#Section2)
3. [Virtual Diffraction Imaging](#Section3)
4. [Machine Learning SPED Data](#Section4)
5. [Peak Finding and Segmentation](#Section5)
6. [Orientation Mapping](#Section6)

## 0. Import pyxem and other required libraries
<a id='Section0'></a>
[To Index](#Index)

In [ ]:
# I like to print out the current version for hyperspy that I am using just in case I come back to a
# notbook a many years later and things change slightly
import hyperspy
print(hyperspy.__version__)
import pyxem
print(pyxem.__version__)

In [ ]:
# Changing the matplotlib background will give you interactive 
%matplotlib qt5
#%matplotlib inline
#%matplotlib widget # for plotting when running remotely on a cluster etc.
import hyperspy.api as hs
import pyxem as pxm
import numpy as np

In [ ]:
# Starting up a distributed Cluster locally 
# You don't have to do this but it helps to visualize what is happening
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

<a id='loa'></a>

##  1. Loading and Inspection
<a id='Section1'></a>
[To Index](#Index)

Load the SPED data acquired from the nano-crystals using hyperspy.

`Note: Because pyxem extends hyperspy this happens automatically!`

In [ ]:
import hyperspy.api as hs
dp = hs.load("data/mgo_nanoparticles.zspy", lazy=True)

In [ ]:
# lets just look at the data 
# display(dp) also works

In [ ]:
# Lets change the title here so it shows up when we load the dataset

In [ ]:
# Then we can display the data strucuture again. 

Inspect the dp object

In [ ]:
# currently the navigator isn't set so in order to plot this we have to "create" one by
# summing the entire dataset. Look at the distributed task-stream to see all of the chunks are
# loaded and then a Summed navigator is created.  This isn't very efficient (or lazy!)


# if you rerun this cell the navigator is saved (Yay) and it takes much less time to plot the data!

Inspect the data type of the object

Inspect the metadata associated with the object 'dp'

Set important experimental parameters using the built in function

In [ ]:
dp.set_experimental_parameters(beam_energy=300.0,
                               camera_length=21.0,
                               scan_rotation=277.0,
                               convergence_angle=0.7,
                               exposure_time=10.0)

See how this changed the metadata

Plot the data to inspect it

## 2. Alignment & Calibration
<a id='Section2'></a>
[To Index](#Index)

Let's center the direct beam for the dataset

In [ ]:
# get the direct beam position using `get_direct_beam_position`


In [ ]:
# compute the shifts


In [ ]:
# plot the orginal shifts


In [ ]:
# make the shifts into a linear plane


In [ ]:
# plot the shifts again!


Align the dataset based on the direct beam position

#### Setting the Calibration
Set the calibration. This is usually known for some detector from a standard sample
or you can get this from the dataset if the scale is known.

In [ ]:
scale = 0.03246
scale_real = 3.03
dp.set_diffraction_calibration(scale)
dp.set_scan_calibration(scale_real)

In [ ]:
# plot calibrated data

<a id='vdf'></a>

##  3. Virtual Diffraction Imaging & Selecting Regions
<a id='Section3'></a>
[To Index](#Index)

### 3.1 Interactive VDF Imaging

Plot an interactive virtual image integrating intensity within a circular subset of pixels in the diffraction pattern

In [ ]:
# create the rois


In [ ]:
# coompute the mean dp


In [ ]:
# lets just add a custom VDF to the image to get both VDF and VBF images


In [ ]:
# inspect the bf roi I like to save these values above for reproduceability.


In [ ]:
# add df roi


In [ ]:
# inspect the Df roi I like to save these values above for reproduceability.


In [ ]:
# get the integrated intensity for the bf and df

Get the virtual diffraction image associated with the last integration window used interactively

In [ ]:
# compute the two images

In [ ]:
# plot both virtual images

In [ ]:
# inspect the metadata

In [ ]:
# save the image

### 3.2 Azimuthal Integration

Pyxem uses the [`pyfai`](https://github.com/silx-kit/pyFAI/tree/v2023.1) library to handle azimuthal integrations including the effects of the Ewald Sphere.  Because of this you should set the calibration and the beam energy before integration.

For speed a AzimuthalIntegrator object is precomputed which reduces redundant calculations.

In [ ]:
# set the unit and beam_energy


In [ ]:
# set the Azimuthal Integrator


In [ ]:
# get the 1D Azimuthal Integration


In [ ]:
# compute the dataset


In [ ]:
# plot the inverse to get the VDF as a function of radius 


### 3.3 Select a region in the scan

Plot the data with an adjustable marker indicating where to crop the scan region

In [ ]:
reg = hs.roi.RectangularROI(left=50.,
                            top=100.,
                            right=100.,
                            bottom=300.)
dp.plot(cmap='inferno')
reg.add_widget(dp)

In [ ]:
# crop the dataset

In [ ]:
# calculate the mean from the selected area

In [ ]:
# plot the mean

## 4. Unsupervised learning
<a id='Section4'></a>
[To Index](#Index)

Perform singular value decomposition (SVD) of the data

Obtain a "Scree plot" by plotting the fraction of variance described by each principal component

In [ ]:
# SVD won't converage with zeros


In [ ]:
# Lazy decompositions only work with the processes and threaded schedulers (not the distributed scheduler)
dpc.compute() # compute this to load it into memory

In [ ]:
# Perform a SVD Decomposition


In [ ]:
# plot_explained_variance_ratio


In [ ]:
# perform NNMF

In [ ]:
# plot decomposition results


<a id='vec'></a>

##  5. Peak Finding
<a id='Section5'></a>
[To Index](#Index)

Perform peak finding on all diffraction patterns in data

In [ ]:
# find peaks in the 2D dataset
# This will immediately compute and return a BaseSignal
peaks = dp.find_peaks(method='difference_of_gaussian',
                       min_sigma=1.,
                       max_sigma=6.,
                       sigma_ratio=1.6,
                       threshold=0.04,
                       overlap=0.99,
                       interactive=False)

Check the peaks object type

In [ ]:
from pyxem.signals import DiffractionVectors

In [ ]:
# Convert the peaks found to a Diffraction Vectors Object
dv = DiffractionVectors.from_peaks(peaks, center=(72, 72), calibration=dp.axes_manager.signal_axes[0].scale)

Look at what's in the peaks object

In [ ]:
# Plot the number of peaks found at each point


In [ ]:
#Cluster the vectors
distance_threshold = 0.1
min_samples = 7
unique_peaks = dv.get_unique_vectors(method='DBSCAN',
                                     distance_threshold=distance_threshold,
                                     min_samples=min_samples)
print(np.shape(unique_peaks.data)[0], ' unique vectors were found.')

In [ ]:
# remove the zero beam
unique_peaks = unique_peaks.filter_magnitude(min_magnitude=.4,
                                   max_magnitude=np.inf)
print(np.shape(unique_peaks)[0], ' unique vectors.')


In [ ]:
# plot the transpose and the unique vectors


In [ ]:
# create Virtual Images from the unique vectors


In [ ]:
#compute the Virtual darkfield images from the peaks

In [ ]:
min_distance = 10.5
min_size = 40
max_size = 1000
max_number_of_grains = 3000
marker_radius = 2
exclude_border = 2
threshold= 0.65

In [ ]:
# test for the right watershed parameters
from pyxem.utils.segment_utils import separate_watershed
i = 21
sep_i = separate_watershed(
    VDFs.inav[i].data, min_distance=min_distance, min_size=min_size,
    max_size=max_size, max_number_of_grains=max_number_of_grains,
    exclude_border=exclude_border, marker_radius=marker_radius,
    threshold=threshold, plot_on=True)

In [ ]:
# get VDF segments
segs = VDFs.get_vdf_segments(min_distance=min_distance,
                             min_size=min_size,
                             max_size = max_size,
                             max_number_of_grains = max_number_of_grains,
                             exclude_border=exclude_border,
                             marker_radius=marker_radius,
                             threshold=threshold)


In [ ]:
# plot the segments


Calculate normalised cross-correlations between all VDF image segments to identify those that are related to the same crystal.

In [ ]:
# get cross correlation matrix

If the correlation value exceeds corr_threshold for certain segments, those segments are summed. These segments are discarded if the number of these segments are below vector_threshold, as this number corresponds to the number of detected diffraction peaks associated with the single crystal. The vector_threshold criteria is included to avoid including segment images resulting from noise or incorrect segmentation.

In [ ]:
corr_threshold=0.3
vector_threshold=4
segment_threshold=2

In [ ]:
corrsegs = segs.correlate_vdf_segments(
    corr_threshold=corr_threshold, vector_threshold=vector_threshold,
    segment_threshold=segment_threshold)
print(np.shape(corrsegs.segments)[0],' correlated segments were found.')

In [ ]:
# plot correlation segments

## 6. Orientation Mapping
<a id='Section6'></a>
[To Index](#Index)

Another way we can look at the different crystals is using orientation mapping.  In this case we take the known crystal strucuture and then compute the kinematic diffraction patterns.  Then we cross correlate each pattern with each diffraction pattern and return the pattern which matches most closely. 

Currently, the algorithm doesn't take into account crystals in projection but that is something that we could add (Or you could add if you are interested!)



In [ ]:
# get a slice of that data
dp_slic = dp.inav[10:50,30:60]

In [ ]:
# Subtract the background using Difference of Gaussians
dp_slic = dp_slic.subtract_diffraction_background(method="difference of gaussians",
                                                  min_sigma=2.5,
                                                  max_sigma=10)

In [ ]:
# Let's look at the subtacted image. 
#dp_slic.plot()

In [ ]:
# import diffpy and diffsims for loading and simulating data
import diffpy
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator

In [ ]:
# Create a grid of roatations
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
resolution = 1.5 # maximum angle in degrees between nearest templates. Pretty rough grid for speed.
grid_cub = get_beam_directions_grid("hexagonal", resolution, mesh="spherified_cube_edge")
print("Number of patterns: ", grid_cub.shape[0])

In [ ]:
# Parameters necessary for simulating a template library
# half size of the images
diffraction_calibration=dp.axes_manager.signal_axes[0].scale
half_shape = (dp.data.shape[-2]//2, dp.data.shape[-1]//2)
# maximum radius in reciprocal space to calculate spot intensities for
reciprocal_radius = np.sqrt(half_shape[0]**2 + half_shape[1]**2)*diffraction_calibration
print("Max Recip. Radius: ", reciprocal_radius, "Inverse Angstroms")

In [ ]:
# simulate the diffraction patterns for MgO
structure_matrix = diffpy.structure.loadStructure("data/MgO.cif")

# "The microscope = the parameters that determine how the templates are calculated"
diff_gen = DiffractionGenerator(accelerating_voltage=300,
                                precession_angle=0,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)

# Generating a library
# "Library of structures and orientations"
library_phases_mgo = StructureLibrary(["MgO"], [structure_matrix], [grid_cub])
# Calculate the actual library

diff_lib_mgo = lib_gen.get_diffraction_library(library_phases_mgo,
                                              calibration=diffraction_calibration,
                                              reciprocal_radius=2,
                                              half_shape=half_shape,
                                              with_direct_beam=False,
                                              max_excitation_error=0.07)

In [ ]:
from pyxem.utils import indexation_utils as iutls

In [ ]:
delta_r = 2 # spacing in r (in pixels)
delta_theta = 1 # spacings in theta (in degrees)
max_r = None # max radius to consider
intensity_transform_function = None # transform the intensity with a function
find_direct_beam = False  # convenience, if the pattern was not centered, this will perform a rough centering
direct_beam_position = (72,72) # direct beam position (centered in pixels)
normalize_image = True # divide the correlation by the norm of the image
normalize_templates = True  # divide the correlation by the norm of the template

In [ ]:
frac_keep = 0.8 
n_keep = None

# if frac_keep < 1 or 1 < n_keep < number of templates then indexation
# templates in "indexes" that have the highest "fast" correlation
n_best = 5 # keep the 5 best rotations
result, phasedict = iutls.index_dataset_with_template_rotation(dp_slic,
                                                    diff_lib_mgo,
                                                    phases = ["MgO"],  # if we have multiple phases we can also specify which ones we want to consider. If it's not specified, all phases are used.
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = 50,
                                                    intensity_transform_function=intensity_transform_function,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    )

In [ ]:
# plotting the orientation maps and the Inverse Pole Figures
solution = result["orientation"]

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from orix.projections import StereographicProjection

# map a vector onto the fundamental zone of the hexagon
def to_fundamental(data_sol):
    data_sol = np.abs(data_sol)
    data_sol = np.sort(data_sol, axis=-1)
    column = data_sol[...,0].copy()
    data_sol[..., 0] = data_sol[...,1]
    data_sol[..., 1] = column
    return data_sol


def get_ipf_color(vectors):
    # the following column vectors should map onto R [100], G [010], B[001], i.e. the identity. So the inverse of 
    # this matrix maps the beam directions onto the right color vector
    color_corners = np.array([[0, 1, 1],
                              [0, 0, 1],
                              [1, 1, 1]])
    color_mapper = np.linalg.inv(color_corners)
    # a bit of wrangling
    data_sol = to_fundamental(vectors.data)
    flattened = data_sol.reshape(np.product(data_sol.shape[:-1]), 3).T
    rgb_mapped = np.dot(color_mapper, flattened)
    rgb_mapped = np.abs(rgb_mapped / rgb_mapped.max(axis=0)).T
    rgb_mapped = rgb_mapped.reshape(data_sol.shape)
    return rgb_mapped
    
    
from orix.quaternion.rotation import Rotation
from orix.vector.vector3d import Vector3d

# draw IPF - Z (row 1), IPF - Y (row 2), IPF - Z (row 3)
fig, ax = plt.subplots(ncols = solution.shape[2], nrows = 3, figsize = (10, 6))

for i in range(solution.shape[2]):
    solution_vectors_z = Rotation.from_euler(np.deg2rad(solution[:,:,i,:]))*Vector3d.zvector()    
    solution_vectors_y = Rotation.from_euler(np.deg2rad(solution[:,:,i,:]))*Vector3d.yvector()    
    solution_vectors_x = Rotation.from_euler(np.deg2rad(solution[:,:,i,:]))*Vector3d.xvector()    
    ax[0, i].set_title(f"Solution {i}")
    ax[0, i].imshow(get_ipf_color(solution_vectors_z))
    ax[1, i].imshow(get_ipf_color(solution_vectors_y))
    ax[2, i].imshow(get_ipf_color(solution_vectors_x))

ax[0,0].set_ylabel("IPF-Z")
ax[1,0].set_ylabel("IPF-Y")
ax[2,0].set_ylabel("IPF-X")

fig.tight_layout()